Вычислить с заданой погрешностью решение задачи Коши

$\left\{ \begin{gathered} \frac{dy}{dx} = -\frac{y^2 + 4x(x + 1)}{y} \\ y(1) = 12 \end{gathered}\right. ,\space \epsilon = 10^{-4}, \space x \in (1, 2)$

In [15]:
import numpy as np
import math

In [16]:
a = 1
b = 2
eps = 1e-4
k = 1
y0 = 12
target = np.linspace(a, b, 11)

def f(x,y):
    if y != 0:
        return (-y**2 + 4*x*(x + 1))/y
    else:
        print('error')
        return 0

In [17]:
def genX(n):
    xx = []
    for i in range(10):
        xx.extend(list(np.linspace(target[i], target[i+1], n, endpoint=False)))

    xx.append(b)
    
    return xx
    
def getStep(xx):
    h = (b-a)/(len(xx)-1)
    return h

In [18]:
def iterate(x, y, h):
    f1 = f(x,y)
    f2 = f(x + h/3, y + h/3*f1)
    f3 = f(x + 2*h/3, y + 2*h/3*f2)
    return y + h/4*(f1 + 3*f3)

In [19]:
def rk(xx, h):
    x = a
    y_prev = y0
    y_curr = y0

    res = []
    for x in xx:
        for t in target:
            if abs(t-x)<1e-8:
                res.append(y_curr)
        y_prev = y_curr
        y_curr = iterate(x, y_curr, h)

        yh = iterate(x, y_curr, h)
        y2h = iterate(x, y_curr, 2*h)
        # assert abs(y2h - yh) <= (2**k - 1) * eps
    return res

In [20]:
xx = genX(4096)
res = rk(xx, getStep(xx))

In [21]:
xxmax = genX(2**20)
resmax = rk(xxmax, getStep(xxmax))

In [22]:
len(xxmax)

10485761